In [17]:
import pandas as pd
import numpy as np
import datetime as dt

from sklearn.preprocessing import LabelEncoder 
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

In [42]:
#Importamos los set de datos.
eventos = pd.read_csv("../events_up_to_01062018.csv", low_memory=False)
labels = pd.read_csv("../trocafone_kaggle_test.csv")

In [43]:
list(eventos)

['timestamp',
 'event',
 'person',
 'url',
 'sku',
 'model',
 'condition',
 'storage',
 'color',
 'skus',
 'search_term',
 'staticpage',
 'campaign_source',
 'search_engine',
 'channel',
 'new_vs_returning',
 'city',
 'region',
 'country',
 'device_type',
 'screen_resolution',
 'operating_system_version',
 'browser_version']

In [89]:
#Creamos un dataframe de features
features = labels

In [109]:
features.shape

(19415, 80)

In [91]:
#Feature: Es un usuario que retorno alguna vez?

df = eventos.groupby(['person'])['new_vs_returning'].agg('count').reset_index()
df['retornoAlSitio'] = df["new_vs_returning"].apply(lambda x: True if x>0 else False)
del df['new_vs_returning']
features = pd.merge(features, df, on='person', how= 'left')


In [92]:
#Feature: Cantidad de tipo de estado de dispositivo que busco.

eventos_visualizacion = eventos.loc[eventos.event == 'viewed product']
df = eventos_visualizacion.groupby('person')['condition'].value_counts().unstack()
df = df.rename(columns={'Bom': 'cantidadVisualizacionesBuenEstado', 
                 'Bom - Sem Touch ID': 'cantidadVisualizacionesBuenEstadoSinTouch', 
                 'Novo': 'cantidadVisualizacionNuevo',
                 'Muito Bom': 'cantidadVisualizacionesMuyBuenEstado',
                 'Excelente': 'cantidadVisualizacionesExcelente'})

df = df.fillna(0)
scaler = MinMaxScaler()
for column in df:
    df[[column]] = scaler.fit_transform(df[[column]])

features = pd.merge(features, df, on='person', how= 'left')

In [93]:
#Feature: Cantidad de eventos generados totales

df = eventos.groupby(['person'])['event'].agg('count').to_frame('cantidad_eventos_generados')
scaler = MinMaxScaler()
for column in df:
    df[[column]] = scaler.fit_transform(df[[column]])

features = pd.merge(features, df, on='person', how= 'left')



In [94]:
#Features: Cantidad por tipo de evento generado
df = eventos.groupby('person')['event'].value_counts().unstack()
df = df.fillna(0)
df = df.rename(columns={'ad campaign hit': 'cantidadIngresosPorCampania', 
                 'brand listing': 'cantidadDeVisualizacionesPorMarca', 
                 'checkout': 'cantidadDeCheckouts',
                 'conversion': 'cantidadConversiones',
                 'generic listing': 'cantidadDeVisualizacionesDeLaHome',
                 'lead': 'cantidadDeGeneracionDeNotificaciones',
                 'search engine hit': 'cantidadDeIngresosPorBuscador',
                 'searched products': 'cantidadDeProductosBuscados',
                 'staticpage': 'cantidadDeVisitas',
                 'viewed product': 'cantidadDeVisualizacionesDeProducto',
                 'visited site': 'cantidadDeIngresosPorUrl',
                })
scaler = MinMaxScaler()
for column in df:
    df[[column]] = scaler.fit_transform(df[[column]])

features = pd.merge(features, df, on='person', how= 'left')

In [95]:
#Features: Cantidad de visualizaciones por marca
eventos_con_marca = eventos.loc[eventos.event == 'viewed product']
eventos_con_marca['marca'] = eventos_con_marca['model'].str.split(' ').str[0]
df = eventos_con_marca.groupby('person')['marca'].value_counts().unstack()
df = df.fillna(0)
df.columns = ['cantidadDeVisualizacionesPor' + str(col) for col in df.columns]
scaler = MinMaxScaler()
for column in df:
    df[[column]] = scaler.fit_transform(df[[column]])
features = pd.merge(features, df, on='person', how= 'left')

C:\Users\felip\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [96]:
#Features: Realizo algun ingreso por tipo de canal proveniente generado
df = eventos.groupby('person')['channel'].value_counts().unstack()
df = df.fillna(0)
df.columns = ['realizoIngresoPorCanal' + str(col) for col in df.columns]

for column in df:
    df[column] = df[column].apply(lambda x: True if x>0 else False)
    
aux = pd.merge(labels, df, on='person', how= 'left')
aux = aux.fillna(False)
features = pd.merge(features, aux, on='person', how='left')



In [97]:
#Features: Realizo algun ingreso por tipo de campania proveniente  generado
df = eventos.groupby('person')['campaign_source'].value_counts().unstack()
df = df.fillna(0)
df.columns = ['realizoIngresoPorCamapania' + str(col) for col in df.columns]

for column in df:
    df[column] = df[column].apply(lambda x: True if x>0 else False)
    
aux = pd.merge(labels, df, on='person', how= 'left')
aux = aux.fillna(False)
features = pd.merge(features, aux, on='person', how='left')



In [98]:
#Features: Si alguna vez realizo una conversion

eventos_conversion = eventos.loc[eventos.event == 'conversion',]
df = eventos_conversion.groupby(['person'])['event'].agg('count').reset_index()
df['realizoConversion'] = df['event'].apply(lambda x: True if x>0 else False)
del df['event']
aux = pd.merge(labels, df, on='person', how= 'left')
#del aux['label']
aux = aux.fillna(False)
features = pd.merge(features, aux, on='person', how='left')


In [99]:
#Features: Si alguna vez realizo un checkout

eventos_conversion = eventos.loc[eventos.event == 'checkout',]
df = eventos_conversion.groupby(['person'])['event'].agg('count').reset_index()
df['realizoCheckout'] = df['event'].apply(lambda x: True if x>0 else False)
del df['event']
aux = pd.merge(labels, df, on='person', how= 'left')
aux = aux.fillna(False)
features = pd.merge(features, aux, on='person', how='left')


In [100]:
#Feature: Realizo algun ingreso por tipo de dispositivo
df = eventos.loc[eventos.event == 'visited site']
df = df.groupby(['person'])['device_type'].value_counts().unstack()

df.columns = ['realizoIngresoPorTipoDeDispositivo' + str(col) for col in df.columns]

for column in df:
    df[column] = df[column].apply(lambda x: True if x>0 else False)
    
aux = pd.merge(labels, df, on='person', how= 'left')
aux = aux.fillna(False)

features = pd.merge(features, aux, on='person', how='left')

In [101]:
#Feature: Es un usuario de brazil?
df = eventos.loc[eventos.event == 'visited site']
df = df.groupby(['person'])['country'].value_counts().unstack()
df['esUnUsuarioDeBrasil'] = df['Brazil'].apply(lambda x: True if x>0 else False)
df = df.loc[:,'esUnUsuarioDeBrasil']
df = df.reset_index()
aux = pd.merge(labels, df, on='person', how= 'left')
aux = aux.fillna(False)
features = pd.merge(features, aux, on='person', how='left')


In [102]:
#Feature: cantidad de busquedas de acuerdo al storage del dispositivo
df = eventos.loc[eventos.event == 'viewed product']
df = df.groupby(['person'])['storage'].value_counts().unstack()
df = df.fillna(0)
df.columns = ['cantidadDeVisualizacionesPorTipoAlmacenamientoDe' + str(col) for col in df.columns]
scaler = MinMaxScaler()
for column in df:
    df[[column]] = scaler.fit_transform(df[[column]])
features = pd.merge(features, df, on='person', how= 'left')

In [103]:
#Feature a que marca realizo la conversion. Dado que la mayoria son iphone, samsung  motorola vamos a armar una columna booleana con esas 3 marcas
eventos_con_marca = eventos
eventos_con_marca['marca'] = eventos['model'].str.split(' ').str[0]
df = eventos_con_marca.loc[eventos.event == 'conversion']
df = df.groupby(['person'])['marca'].value_counts().unstack()
df['realizoConversionAIphone'] = df['iPhone'].apply(lambda x: True if x>0 else False)
df['realizoConversionASamsung'] = df['Samsung'].apply(lambda x: True if x>0 else False)
df['realizoConversionAMotorola'] = df['Motorola'].apply(lambda x: True if x>0 else False)

df = df.loc[:,['realizoConversionAIphone','realizoConversionASamsung', 'realizoConversionAMotorola']]
df = df.reset_index()

aux = pd.merge(labels, df, on='person', how= 'left')

aux = aux.fillna(False)
features = pd.merge(features, aux, on='person', how='left')



In [104]:
#Feature a que marca realizo la checkout. Dado que la mayoria son iphone, samsung  motorola vamos a armar una columna booleana con esas 3 marcas
eventos_con_marca = eventos
eventos_con_marca['marca'] = eventos['model'].str.split(' ').str[0]
df = eventos_con_marca.loc[eventos.event == 'checkout']
df = df.groupby(['person'])['marca'].value_counts().unstack()
df['realizoCheckoutAIphone'] = df['iPhone'].apply(lambda x: True if x>0 else False)
df['realizoCheckoutASamsung'] = df['Samsung'].apply(lambda x: True if x>0 else False)
df['realizoCheckoutAMotorola'] = df['Motorola'].apply(lambda x: True if x>0 else False)
df['realizoCheckoutASony'] = df['Sony'].apply(lambda x: True if x>0 else False)

df = df.loc[:,['realizoCheckoutAIphone','realizoCheckoutASamsung', 'realizoCheckoutAMotorola','realizoCheckoutASony']]
df = df.reset_index()

aux = pd.merge(labels, df, on='person', how= 'left')
aux = aux.fillna(False)
features = pd.merge(features, aux, on='person', how='left')


In [105]:
#Compleamos los campos NA con 0
features = features.fillna(0)

In [106]:
list(features)

['person',
 'retornoAlSitio',
 'cantidadVisualizacionesBuenEstado',
 'cantidadVisualizacionesBuenEstadoSinTouch',
 'cantidadVisualizacionesExcelente',
 'cantidadVisualizacionesMuyBuenEstado',
 'cantidadVisualizacionNuevo',
 'cantidad_eventos_generados',
 'cantidadIngresosPorCampania',
 'cantidadDeVisualizacionesPorMarca',
 'cantidadDeCheckouts',
 'cantidadConversiones',
 'cantidadDeVisualizacionesDeLaHome',
 'cantidadDeGeneracionDeNotificaciones',
 'cantidadDeIngresosPorBuscador',
 'cantidadDeProductosBuscados',
 'cantidadDeVisitas',
 'cantidadDeVisualizacionesDeProducto',
 'cantidadDeIngresosPorUrl',
 'cantidadDeVisualizacionesPorAsus',
 'cantidadDeVisualizacionesPorLG',
 'cantidadDeVisualizacionesPorLenovo',
 'cantidadDeVisualizacionesPorMotorola',
 'cantidadDeVisualizacionesPorQuantum',
 'cantidadDeVisualizacionesPorSamsung',
 'cantidadDeVisualizacionesPorSony',
 'cantidadDeVisualizacionesPoriPad',
 'cantidadDeVisualizacionesPoriPhone',
 'realizoIngresoPorCanalDirect',
 'realizoIngr

In [110]:
features.to_csv("../set_a_predecir-nuevo.csv")

In [108]:
features.sample(30)

,person,retornoAlSitio,cantidadVisualizacionesBuenEstado,cantidadVisualizacionesBuenEstadoSinTouch,cantidadVisualizacionesExcelente,cantidadVisualizacionesMuyBuenEstado,cantidadVisualizacionNuevo,cantidad_eventos_generados,cantidadIngresosPorCampania,cantidadDeVisualizacionesPorMarca,...,cantidadDeVisualizacionesPorTipoAlmacenamientoDe512MB,cantidadDeVisualizacionesPorTipoAlmacenamientoDe64GB,cantidadDeVisualizacionesPorTipoAlmacenamientoDe8GB,realizoConversionAIphone,realizoConversionASamsung,realizoConversionAMotorola,realizoCheckoutAIphone,realizoCheckoutASamsung,realizoCheckoutAMotorola,realizoCheckoutASony
19291,77be8cbc,True,0.000000,0.000000,0.000000,0.000000,0.000000,0.003831,0.003824,0.002115,...,0.000000,0.000000,0.000000,False,False,False,False,False,False,False
11968,3d7e6761,True,0.001008,0.000000,0.000000,0.000000,0.000000,0.000902,0.000000,0.000000,...,0.000000,0.000000,0.000000,False,False,False,False,False,False,False
9255,f63686c5,True,0.000000,0.000000,0.000000,0.000955,0.000000,0.000676,0.001912,0.000000,...,0.000000,0.001458,0.000000,False,False,False,True,False,False,False
1217,f44acd54,True,0.003024,0.000000,0.000948,0.000000,0.000000,0.008564,0.009560,0.000000,...,0.000000,0.000000,0.000000,False,False,False,False,False,True,False
7398,2cbd12ad,True,0.017137,0.016129,0.040758,0.021968,0.000000,0.026144,0.001912,0.004759,...,0.000000,0.014577,0.010610,False,True,False,False,True,False,False
14358,389c7790,True,0.001008,0.048387,0.000948,0.000000,0.000000,0.015551,0.026769,0.008990,...,0.000000,0.000000,0.000000,False,False,False,True,False,False,False
19186,efc2578f,True,0.000000,0.000000,0.000000,0.000955,0.000000,0.000676,0.001912,0.000000,...,0.000000,0.000000,0.000000,False,False,False,False,True,False,False
9075,7d876077,True,0.009073,0.000000,0.001896,0.001910,0.000000,0.009917,0.017208,0.001586,...,0.000000,0.000000,0.000000,False,True,False,False,True,False,False
15383,bcae6812,True,0.002016,0.000000,0.000000,0.000000,0.000000,0.001127,0.000000,0.000000,...,0.000000,0.002915,0.000000,False,False,False,False,True,False,False
7211,93729c43,True,0.021169,0.016129,0.005687,0.009551,0.000000,0.033356,0.063098,0.007932,...,0.018182,0.004373,0.021220,False,True,False,False,True,False,False
